In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import argparse
import sys
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import influence
from mnist_convnet import deepnn
from utils import corrupt_mnist
import copy

np.random.seed(1)

FLAGS = None

# Import data
mnist = corrupt_mnist(input_data.read_data_sets("mnist_data", one_hot=True), 0.03)

# Create the model
with tf.name_scope("net"):
    input_ph = tf.placeholder(tf.float32, [None, 784])
    y = deepnn(input_ph)

# Define loss and optimizer
with tf.name_scope("loss"):
    y_ = tf.placeholder(tf.float32, [None, 10], name="y_target")
    batch_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    cross_entropy = tf.reduce_sum(batch_loss)

lr = tf.Variable(0.1)
opt = tf.train.AdamOptimizer(lr)
grads_and_vars = opt.compute_gradients(cross_entropy)
train_step = opt.apply_gradients(grads_and_vars)

# Test trained model
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver()
sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [2]:
saver.restore(sess, "/home/cgel/deep_quad/mnist_convnet_corrupted.ckpt")

summary_writter = tf.summary.FileWriter("./Hvp_summaries", sess.graph)

In [11]:
test_feed_dic = {input_ph: mnist.test.images, y_: mnist.test.labels}

batch_xs, batch_ys = mnist.train.next_batch(mnist.test.images.shape[0])
train_feed_dic = {input_ph:batch_xs, y_:batch_ys}

testset = (mnist.test.images, mnist.test.labels)
trainset = (mnist.train.images, mnist.train.labels)
#trainset = (mnist.train.images[0:10000], mnist.train.labels[0:10000])

In [4]:
gs = [g for g,v in grads_and_vars if g is not None]

In [13]:
scale = float(len(trainset[0]))
inf = influence.Influence(cross_entropy/scale, input_ph, y_, testset, trainset, cg_iters = 10, dampening=0.001, vervose=3)

error: 0.67208
- iter: 0
Error: 1.02822612068
dad: 0.0277242945939
alpha: 16.2923079551
beta: 2.34063753096
Ad scaling: 0.112184292147
- iter: 1
Error: 1.02216763439
dad: 0.083085312794
alpha: 12.7248602274
beta: 0.98825043612
Ad scaling: 0.0606272499781
- iter: 2
Error: 1.04994340152
dad: 0.0787399306941
alpha: 13.2693382948
beta: 1.05508526661
Ad scaling: 0.0520908520344
- iter: 3
Error: 1.09641953929
dad: 0.0999970529276
alpha: 11.0241382823
beta: 1.09048978855
Ad scaling: 0.0557301173012
- iter: 4
Error: 1.00551728768
dad: 0.134279000272
alpha: 8.95252117724
beta: 0.841057263636
Ad scaling: 0.0571238775805
- iter: 5
Error: 1.05097259117
dad: 0.0860072663044
alpha: 11.7555750381
beta: 1.09245534743
Ad scaling: 0.0467747966135
- iter: 6
Error: 0.959327728597
dad: 0.109571099393
alpha: 10.0806073233
beta: 0.833204159872
Ad scaling: 0.0459060674311
- iter: 7
Error: 1.05686886148
dad: 0.0776691869388
alpha: 11.8491006246
beta: 1.21369057891
Ad scaling: 0.0440309485211
- iter: 8
Error: 1

In [ ]:
#inf.recompute_s(cg_iters = 20, dampening=4e-2, vervose=1)

In [82]:
influences_corrupted = []
influences_non_corrupted = []
grads_corrupted = []
grads_non_corrupted = []
#size = len(mnist.train.labels)
size = 1000
for i in range(size):
    z = (mnist.train.images[i:i+1], mnist.train.labels[i:i+1])
    influ, z_grad = inf.of_and_g(z)
    if mnist.train.corrupted_mask[i]:
        influences_corrupted.append(influ)
        grads_corrupted.append(z_grad)
    else:
        influences_non_corrupted.append(influ)
        grads_non_corrupted.append(z_grad)
all_influences = influences_corrupted + influences_non_corrupted
all_grads = grads_corrupted + grads_non_corrupted

In [83]:
print("non corrupted")
print("mean",np.mean( influences_non_corrupted))
print("std",np.std( influences_non_corrupted))

non corrupted
mean -6.47728385977e-08
std 1.21111189844e-05


In [84]:
print("corrupted")
print("mean",np.mean( influences_corrupted))
print("std",np.std( influences_corrupted))

corrupted
mean -1.88751401152e-06
std 5.31467498935e-06


In [16]:
inf.s = influence.lset(inf.s, 1.)

In [74]:
def train(n, mnist=mnist, learning_rate=0.1e-4):
    sess.run(tf.assign(lr, learning_rate))
    for _ in range(n):
        batch_xs, batch_ys = mnist.train.next_batch(64)
        sess.run(train_step, feed_dict={input_ph: batch_xs, y_: batch_ys})
        
def test():
    test_acc, test_loss = sess.run([accuracy, cross_entropy], feed_dict={input_ph: mnist.test.images,
                                  y_: mnist.test.labels})
    train_acc, train_loss = sess.run([accuracy, cross_entropy], feed_dict=train_feed_dic)
    print("test acc", test_acc)
    print("test loss", test_loss)
    print("train acc", train_acc)
    print("train loss", train_loss) 
    print("---")
    
def testset_loss():
    return sess.run(cross_entropy, feed_dict={input_ph: mnist.test.images, y_: mnist.test.labels})

def train_(n, trainset=mnist, learning_rate=0.1e-4):
    sess.run(tf.assign(lr, learning_rate))
    for _ in range(n):
        batch_xs, batch_ys = mnist.train.next_batch(64)
        sess.run(train_step, feed_dict={input_ph: batch_xs, y_: batch_ys})

In [62]:
# return a new dataset 
def mnist_i(i):
    new_mnist = copy.deepcopy(mnist)
    new_len = new_mnist.train.images.shape[0] -1
    new_images_shape = [new_len] + list(new_mnist.train.images.shape[1:])
    new_labels_shape = [new_len] + list(new_mnist.train.labels.shape[1:])
    new_mnist.train.images.resize(new_images_shape)
    new_mnist.train.labels.resize(new_labels_shape)
    new_mnist.train.images[:] = np.delete(mnist.train.images, i, axis=0)
    new_mnist.train.labels[:] = np.delete(mnist.train.labels, i, axis=0)
    return new_mnist

In [63]:
subset = [1,2,3]

In [70]:
# leave one out retraining 
influence_acc = []
saver.restore(sess, "/home/cgel/deep_quad/mnist_convnet_corrupted.ckpt")
base_testset_loss = testset_loss()
for i in subset:
    z = (mnist.train.images[i:i+1], mnist.train.labels[i:i+1])
    z_influ, z_grad = inf.of_and_g(z)
    new_mnist = mnist_i(i)
    train(100, new_mnist, learning_rate=1e-6)
    d_loss = base_testset_loss - testset_loss()
    influence_acc.append( (z_influ, d_loss) )
    saver.restore(sess, "/home/cgel/deep_quad/mnist_convnet_corrupted.ckpt")

In [71]:
for pair in influence_acc:
    print(pair[0], pair[1])

7.37678829755e-14 1.04041
-2.86043394536e-11 1.04028
5.96473021503e-15 1.04053


In [75]:
test()
for i in range(3):
    train(1000, 1e-6)
    test() 

test acc 0.9659
test loss 1377.63
train acc 1.0
train loss 4.47512
---


AttributeError: 'float' object has no attribute 'train'

In [5]:
test()
for i in range(5):
    train(10000)
    test()
print("XXXX")
for i in range(3):
    train(10000, 1e-5)
    test() 

test acc 0.0719
train acc 0.0783
---
test acc 0.9624
train acc 0.9539
---
test acc 0.9695
train acc 0.9694
---
test acc 0.9706
train acc 0.9764
---
test acc 0.9683
train acc 0.9873
---
test acc 0.9671
train acc 0.9958
---
XXXX
test acc 0.9682
train acc 0.999
---
test acc 0.9667
train acc 0.9998
---
test acc 0.9672
train acc 0.9993
---


In [42]:
for i in range(2):
    train(10000, 1e-5)
    test() 

test acc 0.9655
train acc 1.0
---
test acc 0.9651
train acc 1.0
---


In [43]:
for i in range(2):
    train(10000, 1e-6)
    test() 

test acc 0.9662
train acc 1.0
---
test acc 0.9659
train acc 1.0
---


In [45]:
gs_np, loss = sess.run([gs, cross_entropy], feed_dict=train_feed_dic)
print("train")
print("grad norm",influence.lnorm(gs_np))
print("loss", loss)

train
grad norm 0.00566148720883
loss 0.000413698


In [46]:
gs_np, loss = sess.run([gs, cross_entropy], feed_dict=test_feed_dic)
print("test")
print("grad norm",influence.lnorm(gs_np))
print("loss", loss)

test
grad norm 0.672080902145
loss 0.137763


In [10]:
saver.save(sess, "/home/cgel/deep_quad/mnist_convnet_corrupted.ckpt")

'/home/cgel/deep_quad/mnist_convnet_corrupted.ckpt'

In [79]:
np.arange(0, 102, 10)

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100])

In [ ]:
mnist.train.

In [7]:
inf.power_iteration(iters=100, epsilon=0.0000001)

------
i: 0
e_value: 0.00209694207342
d: 0.000463992465079
------
i: 1
e_value: 0.208728579166
d: 0.125567949542
------
i: 2
e_value: 0.324813067168
d: 0.300162498384
------
i: 3
e_value: 0.359543528431
d: 0.411048534363
------
i: 4
e_value: 0.374532068351
d: 0.478665687999
------
i: 5
e_value: 0.381948688661
d: 0.516788905821
------
i: 6
e_value: 0.386042167284
d: 0.534752966201
------
i: 7
e_value: 0.388610587858
d: 0.538380275956
------
i: 8
e_value: 0.390461300743
d: 0.53107601776
------
i: 9
e_value: 0.391975541989
d: 0.514732760272
------
i: 10
e_value: 0.39335060605
d: 0.490329457008
------
i: 11
e_value: 0.394694985212
d: 0.458311550666
------
i: 12
e_value: 0.396076195252
d: 0.418841258751
------
i: 13
e_value: 0.39753462587
d: 0.371977643307
------
i: 14
e_value: 0.399097710709
d: 0.31781342407
------
i: 15
e_value: 0.400778156071
d: 0.256585544159
------
i: 16
e_value: 0.40257995568
d: 0.188761466421
------
i: 17
e_value: 0.404494614403
d: 0.115096336608
------
i: 18
e_value

0.43241379468192576

In [8]:
mnist.train.c_labels = mnist.train.labels.copy()

In [10]:
mnist.train.labels[:] = np.zeros_like(mnist.train.labels)

In [13]:
mnist.train.next_batch(1)[1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])